In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import math as m
from scipy.optimize import curve_fit
import geopandas as geo
import os
import gdal
import osr

from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
plt.style.use('classic')

def g_d(lat1,lon1,lat2,lon2):
    R     = 6371.009
    lat1  = lat1*np.pi/180
    lat2  = lat2*np.pi/180
    lon1  = lon1*np.pi/180
    lon2  = lon2*np.pi/180
    A     = np.sin(lat1)*np.sin(lat2)+np.cos(lat1)*np.cos(lat2)*np.cos(lon1-lon2)
    A[A>1]= 1
    return R*np.arccos(A)

def AA_spatialtemporalsemig(data,S_LagNum,S_Lagleng,T_LagNum,T_Lagleng,s_m):
    semig      = np.zeros((T_LagNum,S_LagNum))
    semigCount = np.zeros((T_LagNum,S_LagNum))
    NN         = len(data[:,0])
    for i in range(NN-1):
        k1 = data[i,3]
        k2 = np.where((s_m>=k1)&(s_m<k1+int(T_LagNum*T_Lagleng)))[0][-1]
        k  = np.where(data[:,3]==s_m[k2])[0][-1]
        
        vector    = data[i+1:k+1,:]
        distSpat  = g_d(data[i,0],data[i,1],vector[:,0],vector[:,1])
        distTemp  = np.abs(data[i,3]-vector[:,3])
        indxSpat0 = np.array(distSpat/S_Lagleng,dtype=np.int)+1
        indxTemp0 = np.array(distTemp/T_Lagleng,dtype=np.int)+1

        cond2     = (indxSpat0<=S_LagNum)&(indxTemp0<=T_LagNum)
        vector2   = vector[cond2,:]
        indxSpat2 = indxSpat0[cond2]
        indxTemp2 = indxTemp0[cond2]
        distSpat2 = distSpat[cond2]
        distTemp2 = distTemp[cond2]

        for j in range(len(indxTemp2)):
            semigCount[indxTemp2[j]-1,indxSpat2[j]-1]= semigCount[indxTemp2[j]-1,indxSpat2[j]-1]+1
            semig[indxTemp2[j]-1,indxSpat2[j]-1]    = semig[indxTemp2[j]-1,indxSpat2[j]-1]+(data[i,2]-vector2[j,2])**2
    
    semig=semig/(semigCount*2)
    return semig,semigCount

def AA_CressieIterativeWLS_STsemig(ST_semig,ST_semigCount,start):
    nt = ST_semig.shape[0]
    ns = ST_semig.shape[1]
    
    semig      = np.zeros((nt*ns,4))
    semig[:,0] = ST_semigCount.reshape(nt*ns,order='F')
    semig[:,3] = ST_semig.reshape(nt*ns,order='F')
    semig[0,0] = semig[0,0]*10. # 10 or 5 or 1 
    
    for i in range (ns):
        semig[((i+1)*nt-nt):(i+1)*nt,1] = (i+1)*100-50
        semig[((i+1)*nt-nt):(i+1)*nt,2] = np.arange(4.5,10*nt,10) 
    
    hs = semig[:,1];
    ht = semig[:,2];
    
    def f_hs(ks,hs): 
        return ks[0]*(1.-np.exp((-1.)*(hs/ks[1])))
    def f_ht(kt,ht):
        return kt[0]*(1.-np.exp((-1.)*(ht/kt[1])))
    def modelFun(x,k): 
        return f_hs(k[:2],x[0]) + f_ht(k[2:4],x[1]) - k[4]*f_hs(k[0:2],x[0])*f_ht(k[2:4],x[1])+k[5]
    
    x  = np.array([hs,ht])
    ww = np.sqrt(semig[:,0])/modelFun(x,start)
    yw = ww*semig[:,3]
    
    def modelFunw(x,k1,k2,k3,k4,k5,k6):
        k = np.array([k1,k2,k3,k4,k5,k6])
        return ww * modelFun(x,k)
    
    #[bFitw] = nlinfit(x,yw,modelFunw,start)
    bFitw,_ = curve_fit(modelFunw,x,yw,start)
    
    i  =1
    ee =1e-8
    while np.sum((bFitw-start)**2)>ee :
        start = bFitw
        ww    = np.sqrt(semig[:,0])/modelFun(x,start)
        yw    = ww * semig[:,3]
        #def modelFunw(k,x):
        #    return ww * modelFun(k,x)
        bFitw,_ = curve_fit(modelFunw,x,yw,start)
        i     = i + 1
    
    W_theta     = np.sum(semig[:,0] * ((semig[:,3] / modelFun(x,bFitw)) - 1.) ** 2)
    OutputSemig = np.array([hs,ht,modelFun(x,bFitw)])
    InputSemig  = semig[:,[1,2,3]]
    return bFitw,i,InputSemig,OutputSemig

def myfun(hs,ht,bFitw):
    ######Product-Sum + Global Nugget#######################
    def f_hs(ks,hs): 
        return ks[0]*(1.-np.exp((-1.)*(hs/ks[1])))
    def f_ht(kt,ht):
        return kt[0]*(1.-np.exp((-1.)*(ht/kt[1])))
    def modelFun(x,k): 
        return f_hs(k[:2],x[0]) + f_ht(k[2:4],x[1]) - k[4]*f_hs(k[0:2],x[0])*f_ht(k[2:4],x[1])+k[5]
    ########################################################
    
    # f_hs=@(ks,hs) ks(1)*(1-exp((-1)*hs./ks(2)));
    # f_ht=@(kt,ht) kt(1)*(1-exp((-1)*(ht.^2)/kt(2)));
    # modelFun=@(k,x) f_hs(k(1:2),x(:,1))+f_ht(k(3:4),x(:,2))-k(5)*f_hs(k(1:2),x(:,1)).*f_ht(k(3:4),x(:,2))+k(6);
    x  = np.array([hs,ht])
    return modelFun(x,bFitw)

def SpatialTemporalKrigPrediction_ACOS(ObservedData,WaitForKrig,bFitw):
    #ObservedData [lat,lon,co2,day],
    #fun myfun(hs,ht),
    #WaitForKrig [lat,lon,day]
    #lastValue PredVar, prediction variance

    N = ObservedData.shape[0]
    v = np.ones((N+1,N+1))
    y = np.zeros((N+1,1))
    vstar = np.ones((N+1,1))
    
    for i in range (N):
        v[i,:N] = myfun(g_d(ObservedData[i,0],ObservedData[i,1],ObservedData[:,0],ObservedData[:,1]),
                        np.abs(ObservedData[i,3] - ObservedData[:,3]),bFitw)
        

    v = np.real(v)
    np.fill_diagonal(v,0)
    y[:N,0] = ObservedData[:,2]

    hs = g_d(WaitForKrig[0],WaitForKrig[1],ObservedData[:,0],ObservedData[:,1])
    ht = np.abs(WaitForKrig[2] - ObservedData[:,3])
    vstar[:N,0] = myfun(hs,ht,bFitw)
    
    tem     = np.matmul(np.linalg.inv(v), y)#tem=v\y' #the same as inv(v)*y; result is column vector
    
    lastVal = np.matmul(vstar.T,tem)
    PredVar = np.matmul(vstar.T,np.matmul(np.linalg.inv(v),vstar))
                        
    return lastVal,PredVar  

def SpatialTemporalKriging_ACOS(ObservedData,PredPos,bFitw):
    #[ObservedData]= [lat,lon,co2,day3], is the global ACOS data except the PredPos
    #[PredPos]= [lat,lon,day], 
    #TempTrend [482,1] 482 time units
    #Fitw [6 coefficients]
    ############tic
    NN = PredPos.shape[0]
    KrigSearchNumRadius = np.zeros((NN,3))
    KrigResult = np.zeros((NN,1))
    PredictVar = np.zeros((NN,1))
    for i in range (NN):     
        #krigData=PredPos(i,:)
        #find the points
        distSpatial  = g_d(PredPos[i,0],PredPos[i,1],ObservedData[:,0],ObservedData[:,1])
        distTemporal = np.abs(PredPos[i,2] - ObservedData[:,3])
    
        a    = 300
        b    = 1 #different from IEEE TGRS, because more data available, original [300,20]
        indx = (distSpatial<=a)&(distTemporal<=b)

        while np.sum(len(indx[np.where(indx==True)]))<20 :
            a    = a + 100 #expand until at least 20 data points are available
            b    = b + 1
            indx = (distSpatial<=a)&(distTemporal<=b)

        KrigSearchNumRadius[i,:] = np.array([np.sum(len(indx[np.where(indx==True)])),a,b])
        ObservedData2            = ObservedData[indx,:]
        
        lastVal,PredVar = SpatialTemporalKrigPrediction_ACOS(ObservedData2,PredPos[i,:],bFitw) 
        KrigResult[i]   = lastVal
        PredictVar[i]   = PredVar
          
    return KrigResult,KrigSearchNumRadius,PredictVar

def STK_Mapping_ACOS(ACOSDataRes,bFitw,TempTrend,SpatialTrendPara,InterpGrids):
    # Mapping Northern China with uniform trend
    # Feb 27, 2016
    # ACOSData3 is the Northern China residual data [lat,lon,XCO2res,Day3]
    # TempTrend [122 time-units], SpatialTrendPara [b0,b1,b2] which are fitted surface parameters
    # InterpGrids [lat,lon], grids to be interpolated
    
    NN = InterpGrids.shape[0]
    
    MapKrigResult = np.zeros((NN,n_m))
    MapKrigNum    = np.zeros((NN,n_m))
    MapKrigSpaRad = np.zeros((NN,n_m))
    MapKrigTemRad = np.zeros((NN,n_m))
    MapPredictVar = np.zeros((NN,n_m))
    PredPos       = np.zeros((NN,3))

    # Spatial Trend
    SpatialTrend2 = (np.ones((NN)) * SpatialTrendPara[0] 
                     + InterpGrids[:,0] * SpatialTrendPara[1]
                     + InterpGrids[:,1] * SpatialTrendPara[2])
    
    for j in range (n_m) :
        PredPos[:,0] = InterpGrids[:,0]
        PredPos[:,1] = InterpGrids[:,1]
        PredPos[:,2] = j+1
        KR,KNumKSRadKTRad,PredVar = SpatialTemporalKriging_ACOS(ACOSDataRes,PredPos,bFitw)
        KR         = KR[:,0]
        PredVar    = PredVar[:,0]
        TempTrend2 = TempTrend[j]
        KR = KR + SpatialTrend2
        KR = KR + TempTrend2
        
        MapKrigResult[:,j] = KR
        MapKrigNum[:,j]    = KNumKSRadKTRad[:,0]
        MapKrigSpaRad[:,j] = KNumKSRadKTRad[:,1]
        MapKrigTemRad[:,j] = KNumKSRadKTRad[:,2]
        MapPredictVar[:,j] = PredVar
    return MapKrigResult,MapKrigNum,MapKrigSpaRad,MapKrigTemRad,MapPredictVar

def func1(x, a,b,c):
    return a + b*x[:,0] + c*x[:,1]

def func(x, a0,a1,a2,a3,a4,a5):
    omega= (2*np.pi)/TT
    sin = a2 * np.sin(omega * x) + a4 * np.sin(2*omega*x)
    cos = a3 * np.cos(omega * x) + a5 * np.cos(2*omega*x)
    return a0 + a1 * x  + sin + cos

def CreateFolder(directory):
    try :
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory.' + directory)


data_v =  np.loadtxt('./Grid_oco2_co2_middle_east_1.txt',dtype=[('lat','f8'),('lon','f8'),
                                                        ('time','f8'),('co2','f8'),('var','f8')])

add = './result_go/'
CreateFolder(str(add))

a      = np.zeros((len(data_v['time']),4))
a[:,0] = data_v['lat']
a[:,1] = data_v['lon']
a[:,2] = data_v['time']
a[:,3] = data_v['co2']

s_m = np.unique(a[:,2])
n_m = len(s_m)
TT  = 12

#step1
#removing spatial trend
BB, pcov = curve_fit(func1, a, a[:,3])
s_trend  = func1(a,*BB)
spa_res  = a[:,3] - s_trend
print('step1')

#step 2
#removing Ttiming Trend
spa_res_tmean = np.zeros((1,n_m))
for i,l in enumerate(s_m):
    spa_res_tmean[0,i] = np.mean(spa_res[a[:,2]==l])

params2, pcov2 = curve_fit(func, s_m, spa_res_tmean[0])
spa_res_t_res  = np.zeros(spa_res.shape)
time_t         = func(s_m,*params2)
for i,l in enumerate(s_m):
    spa_res_t_res[a[:,2]==l] = spa_res[a[:,2]==l] - time_t[i]

    
plt.figure()
plt.plot(spa_res_tmean[0],'o')
plt.plot(func(s_m,*params2))
plt.savefig('./'+str(add)+'/fit.pdf')
plt.close()

data      = np.zeros((a.shape))
data[:,0] = a[:,0]
data[:,1] = a[:,1]
data[:,3] = a[:,2]
data[:,2] = spa_res_t_res
print('step2')

#step 3
try :
    CHNsemig      = np.loadtxt('./'+str(add)+'/CHNsemig.txt')
    CHNsemigCount = np.loadtxt('./'+str(add)+'/CHNsemigCount.txt')
except :
    CHNsemig,CHNsemigCount = AA_spatialtemporalsemig(data,20,100,10,1,s_m)
    np.savetxt('./'+str(add)+'/CHNsemig.txt',CHNsemig)
    np.savetxt('./'+str(add)+'/CHNsemigCount.txt',CHNsemigCount)
print('step3')

##Step 4
#Spatiotemporal variogram modeling
#Using Product-Sum model by De Iaco et al. (2001)

T   = 8
S   = 14
sta =  np.array([2.,1000.,1.5,5.,0.1,1.])
t_bFitw,ii,t_InputSemig,t_OutputSemig = AA_CressieIterativeWLS_STsemig(CHNsemig[:T,:S],CHNsemigCount[:T,:S],sta)
print('step4')

#step 5
tLat        = np.arange(11,44,0.5)
tLon        = np.arange(23,65,0.5)
tPlg,tPlt   = np.meshgrid(tLon,tLat)
InterpGrids = np.array([tPlt,tPlg]).reshape(2,tLat.size*tLon.size,order='F').T #cordinates of map
try:
    MapKrigResult = np.loadtxt('./'+str(add)+'/MapKrigResult')
    MapKrigNum    = np.loadtxt('./'+str(add)+'/MapKrigNum')
    MapKrigSpaRad = np.loadtxt('./'+str(add)+'/MapKrigSpaRad')
    MapKrigTemRad = np.loadtxt('./'+str(add)+'/MapKrigTemRad')
    MapPredictVar = np.loadtxt('./'+str(add)+'/MapPredictVar')
except:
    MapKrigResult,MapKrigNum,MapKrigSpaRad,MapKrigTemRad,MapPredictVar=STK_Mapping_ACOS(data,t_bFitw,time_t,BB,
                                                                                        InterpGrids)
    np.savetxt('./'+str(add)+'/MapKrigResult',MapKrigResult)
    np.savetxt('./'+str(add)+'/MapKrigNum',MapKrigNum)
    np.savetxt('./'+str(add)+'/MapKrigSpaRad',MapKrigSpaRad)
    np.savetxt('./'+str(add)+'/MapKrigTemRad',MapKrigTemRad)
    np.savetxt('./'+str(add)+'/MapPredictVar',MapPredictVar)
print('step5')

# Step 6
# Visulize the mapping data
CreateFolder('./'+str(add)+'/map/')
for j,l in enumerate(s_m):
    tdata    = MapKrigResult[:,j]
    tGridMap = np.reshape(tdata,(66,84),order='F')
    nrows,ncols  = np.shape(tGridMap)
    xmin,ymin,xmax,ymax = [tLon.min(),tLat.min(),tLon.max(),tLat.max()]

    xres = (xmax-xmin)/float(ncols)
    yres = (ymax-ymin)/float(nrows)


    geotransform = (xmin, xres,0,ymax,0, -yres) 
    output_raster = gdal.GetDriverByName('GTiff').Create('./'+str(add)+'/map/'+
                                                         str(2015+int(l-1)//12)+'_'+str(np.mod(int(l-1),12)+1)+'.tif',
                                                         ncols, nrows, 1 ,gdal.GDT_Float32)
    output_raster.SetGeoTransform(geotransform)

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    output_raster.SetProjection( srs.ExportToWkt() ) 

    output_raster.GetRasterBand(1).WriteArray(np.flip(np.rot90(tGridMap).T))
    output_raster.FlushCache()
print('step6')
print('part '+str(i+1))
print('---------------------')

step1
step2
step3
step4
step5
step6
part 71
---------------------


['/Users/saeedansarifard/Work/Mohit_zist/Kriging_new/d_oco2', '/opt/anaconda3/lib/python38.zip', '/opt/anaconda3/lib/python3.8', '/opt/anaconda3/lib/python3.8/lib-dynload', '', '/opt/anaconda3/lib/python3.8/site-packages', '/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/saeedansarifard/.ipython']
